In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [2]:
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN,Dense

In [3]:
plays=pd.read_csv("/content/Shakespeare_data.csv")
plays.shape
plays.columns

Index(['Dataline', 'Play', 'PlayerLinenumber', 'ActSceneLine', 'Player',
       'PlayerLine'],
      dtype='object')

In [4]:
samples_lines=plays['PlayerLine'].sample(n=100,random_state=42)
data=" ".join(samples_lines)

In [5]:
data

"That hath deprived me of your grace and favour, Their bodies, even to loathing, for they so stunk, Men at some time are masters of their fates: Disgorges such a tempest forth, That monster, custom, who all sense doth eat, To this chair bind him. Villain, thou shalt find-- Dexterity so obeying appetite Hector, in view of Trojans and of Greeks, Who know the world, see heaven, but, feeling woe, I should my tears let fall upon your cheek, Let not that doctor e'er come near my house: Arise, and say how thou camest here. Have I not heard these islanders shout out and one thing more, that you be never so hardy to And say I am Revenge, sent from below To furnish me upon my longing journey. All his revenue. And thus the native hue of resolution Why, art thou mad, old fellow? porringer fell off her head, for kindling such a That you shall stifle in your own report Is not this suit of mine, that thou declare It is as easy to count atomies as to resolve the But let this same be presently perform'

In [6]:
#tokenization
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
word_index=tokenizer.word_index
total_words=len(word_index)+1

In [7]:
input_sequences=[]
token_list=tokenizer.texts_to_sequences([data])[0]
for i in range(1,len(token_list) ):
  n_gram_sequence=token_list[:i+1]
  input_sequences.append(n_gram_sequence)

In [8]:
max_sequence_len=max([len(seq) for seq in input_sequences])
input_sequences=pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre')

In [9]:
X,y=input_sequences[:,:-1],input_sequences[:,-1]
y=to_categorical(y,num_classes=total_words)

In [10]:
model=Sequential()
model.add(Embedding(total_words,1000,input_length=max_sequence_len-1))
model.add(SimpleRNN(200))
model.add(Dense(total_words,activation='softmax'))

In [11]:
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 751, 1000)         387000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 200)               240200    
                                                                 
 dense (Dense)               (None, 387)               77787     
                                                                 
Total params: 704987 (2.69 MB)
Trainable params: 704987 (2.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.fit(X,y,epochs=100,batch_size=64,verbose=1)

Epoch 1/100
12/12 [==============================] - 14s 819ms/step - loss: 5.9450 - accuracy: 0.0120
Epoch 2/100
12/12 [==============================] - 9s 771ms/step - loss: 5.5471 - accuracy: 0.0706
Epoch 3/100
12/12 [==============================] - 7s 590ms/step - loss: 5.1527 - accuracy: 0.1704
Epoch 4/100
12/12 [==============================] - 8s 709ms/step - loss: 4.7774 - accuracy: 0.2490
Epoch 5/100
12/12 [==============================] - 8s 669ms/step - loss: 4.3731 - accuracy: 0.3369
Epoch 6/100
12/12 [==============================] - 8s 591ms/step - loss: 3.9621 - accuracy: 0.4541
Epoch 7/100
12/12 [==============================] - 8s 695ms/step - loss: 3.5220 - accuracy: 0.5952
Epoch 8/100
12/12 [==============================] - 15s 1s/step - loss: 3.0715 - accuracy: 0.7417
Epoch 9/100
12/12 [==============================] - 9s 778ms/step - loss: 2.6285 - accuracy: 0.8402
Epoch 10/100
12/12 [==============================] - 7s 574ms/step - loss: 2.1934 - accurac

In [13]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [14]:
seed_text = "That hath deprived me of your"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Next word prediction: {next_word}")

Next word prediction: grace


In [15]:
seed_text = "To furnish me upon my longing"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Next word prediction: {next_word}")

Next word prediction: journey


In [16]:
seed_text = "Dexterity so obeying appetite"
next_word = predict_next_word(model, tokenizer, seed_text, max_sequence_len)
print(f"Next word prediction: {next_word}")

Next word prediction: hector
